In [2]:
import cv2 as cv
import numpy as np

In [3]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
   
    slope_lines = []
    intercept_lines = []
    slope_left = []
    slope_right = []
    intercept_left = []
    intercept_right = []
    img_y_size = img.shape[0]
    slope_left_sign = []
    slope_right_sign = []

    
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope_ind = (y2-y1)/(x2-x1) 
            intercept_ind = y1 - x1 * ((y2-y1)/(x2-x1))
            slope_lines.append(slope_ind)
            intercept_lines.append(intercept_ind) 

    for val in slope_lines:
        if val < 0:
            slope_left_sign.append(val)
        else:
            slope_right_sign.append(val)

    mean_slope_left = np.mean(slope_left_sign)
    std_slope_left = np.std(slope_left_sign)
    mean_slope_right = np.mean(slope_right_sign)
    std_slope_right = np.std(slope_right_sign)
    
    for s,i in zip(slope_lines, intercept_lines):
        if abs(s-mean_slope_left) < std_slope_left: 
            slope_left.append(s)
            intercept_left.append(i)
        elif abs(s-mean_slope_right) < std_slope_right:
            slope_right.append(s)
            intercept_right.append(i)

    if len(slope_left) > 0:
        left_line_para = [sum(slope_left)/len(slope_left), sum(intercept_left)/len(intercept_left)]
        left_y1 = int(img_y_size)
        left_x1 = int((left_y1 - left_line_para[1])/ left_line_para[0])
        left_y2 = int(img_y_size * 0.580)
        left_x2 = int((left_y2 - left_line_para[1])/ left_line_para[0])
        cv.line(img, (left_x1, left_y1), (left_x2, left_y2), color, 10)

    if len(slope_right) > 0:
        right_line_para = [sum(slope_right)/len(slope_right), sum(intercept_right)/len(intercept_right)]
        right_y1 = int(img_y_size)
        right_x1 = int((right_y1 - right_line_para[1])/ right_line_para[0])
        right_y2 = int(img_y_size * 0.580)
        right_x2 = int((right_y2 - right_line_para[1])/ right_line_para[0])
        cv.line(img, (right_x1, right_y1), (right_x2, right_y2), color, 10)

In [4]:
def region_of_interest(img, vertices):
    # blank mask to start
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv.bitwise_and(img, mask)
    return masked_image

In [5]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        draw_lines(line_img, lines)
    return line_img

In [6]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv.addWeighted(initial_img, α, img, β, γ)


In [7]:
def process_image(image):

    #gray scale conversion
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)

    #hsv image
    hsv_img = cv.cvtColor(image, cv.COLOR_RGB2HSV)

    #range of color in HSV image
    lower_range_y = np.array([20,100,100])
    upper_range_y = np.array([30,255,255])
    lower_range_w = np.array([0,0,235])
    upper_range_w = np.array([255,255,255])

    # extract yellow and white part from the image
    yellow_part = cv.inRange(hsv_img, lower_range_y, upper_range_y)
    white_part = cv. inRange(hsv_img, lower_range_w, upper_range_w)
    whole_mask = cv.bitwise_or(yellow_part, white_part)

    # brightness reduction of gray image 
    lower_brightness_gray = (gray / 2).astype('uint8')

    #boosted image 
    boost_lane = cv.bitwise_or(lower_brightness_gray, whole_mask)

    #gaussian blur
    blur_img = cv.GaussianBlur(boost_lane,(5,5),0)

    # canny edge detection apply
    edges = cv.Canny(blur_img, 60, 150)

    # masked edges 
    vertices = np.array([[(50, 540), (460,310), (500,310), (960,540)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # hough lines
    color_edges = hough_lines(masked_edges, 3, np.pi/180, 40, 70,250)
    # hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap)

    # drawing line on edges image
    result =  weighted_img(color_edges, image)
    
    return result

In [10]:
video_capture = cv.VideoCapture("test_videos\solidYellowLeft.mp4")

# Get the video's frames per second (fps) and frame size
fps = video_capture.get(cv.CAP_PROP_FPS)
frame_size = (int(video_capture.get(3)), int(video_capture.get(4)))

# Create VideoWriter object to save the processed video
fourcc = cv.VideoWriter_fourcc(*"mp4v")  
output_video = cv.VideoWriter("output_video.mp4", fourcc, fps, frame_size)

while True:
    # Read the next frame from the video
    ret, frame = video_capture.read()

    # Break the loop if the video has ended
    if not ret:
        break

    # Process the frame using your image processing pipeline
    processed_frame = process_image(frame)

    # Display the processed frame
    cv.imshow('Processed Frame', processed_frame)

    # Write the processed frame to the output video file
    output_video.write(processed_frame)

    # Break the loop if 'q' key is pressed
    if cv.waitKey(1) & 0xFF == ord('c'):
        break

# Release the video capture and writer objects
video_capture.release()
output_video.release()
cv.destroyAllWindows()